# Initialise the workspace 
In order to deploy the workspace needs to initialised from the config file

In [2]:
from azureml.core.authentication import InteractiveLoginAuthentication
forced_interactive_auth = InteractiveLoginAuthentication(tenant_id="ca3fdb0e-9d68-48a6-8b66-2ae9e3cb6e2f", force=True)

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


KeyboardInterrupt: 

In [3]:
from azureml.core import Workspace
ws = Workspace.from_config(path=".azureml/config_ws.json")
print(ws.name, ws.resource_group, ws.location, sep ='\n')

af-detection
af-detection
australiaeast


# Register the Model to workspace
Take the local file and register the model the azure workspace

In [3]:
from azureml.core.model import Model

model = Model.register(workspace=ws, 
                    model_path="./ModelTrainedOnCambellSamples.h5", 
                    model_name="af-detection-Model")
print(model)

Registering model af-detection-Model
Model(workspace=Workspace.create(name='af-detection', subscription_id='e878668b-9e6d-46af-b058-d4285cdf00b1', resource_group='af-detection'), name=af-detection-Model, id=af-detection-Model:2, version=2, tags={}, properties={})


# Create environment for deployment
Creates the deployment environment with dependencies and modules

In [4]:
import tensorflow
from azureml.core.environment import Environment

environment = Environment("Local Deploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("tensorflow==2.4.1")
environment.python.conda_dependencies.add_pip_package("scipy")



In [5]:
import os

source_directory = "source_dir"

os.makedirs(source_directory, exist_ok=True)

# Create entry script for Webservice
This script is used to initialise the ML service and handle its runtime behaviour when called

In [4]:
%%writefile source_dir/score.py
import json
import numpy as np 
from tensorflow.keras.models import load_model
import process_ecg
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("af-detection-Model")
    model = load_model(model_path)

def run(request):
    try:
        data = json.loads(request)
        ecg_array = np.array(data)
        processed = process_ecg.process(ecg_array)
        result = model.predict(processed)
        prediction = np.argmax(result)+1
        reject = 0
        if np.max(result[0]) < 0.97 and (np.max(result[0])-process_ecg.second_largest(result[0])) < 0.95:
            reject = 1
        return (int(Prediction), int(reject), model_name)
    except Exception as e:
        error = str(e)
        return {'data' : error, "message" : 'unable to classify sample'}


Overwriting source_dir/score.py


# Create the configuration for inference
This is to provide the webservice of knowledge of what environment to and what entry script to run

In [18]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(environment=environment, source_directory=source_directory, entry_script='score.py')

# Deploy the model as Local Webservice with Docker

Using a docker container deploy the local service. 

In [19]:
from azureml.core.webservice import LocalWebservice 

deployment_config = LocalWebservice.deploy_configuration(port=6789)
local_service = Model.deploy(ws, 'test', [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 0a6a6858b31d47ee9c7b2bfdb0b97579.azurecr.io
Logging into Docker registry 0a6a6858b31d47ee9c7b2bfdb0b97579.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 0a6a6858b31d47ee9c7b2bfdb0b97579.azurecr.io/azureml/azureml_20ac39522139982557aec7a98440dfb6
 ---> 820b6dbed4ab
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 4d3329296019
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImU4Nzg2NjhiLTllNmQtNDZhZi1iMDU4LWQ0Mjg1Y2RmMDBiMSIsInJlc291cmNlR3JvdXBOYW1lIjoiYWYtZGV0ZWN0aW9uIiwiYWNjb3VudE5hbWUiOiJhZi1kZXRlY3Rpb24iLCJ3b3Jrc3BhY2VJZCI6IjBhNmE2ODU4LWIzMWQtNDdlZS05YzdiLTJiZmRiMGI5NzU3OSJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in a3ff85219ace
 ---> 7f983e95c698
Step 4/5 : RUN mv '/var/azureml-app/tmpxfiw_od2.py' /var/azureml-app/main.py
 ---> Running in 238d17bba31

# Test the Webservice using sample data
Load Sample ecg signals and test the local deployment classification

In [5]:
import json
import numpy as np

sample_ecg = np.fromfile("./source_dir/ecg_sample.dat", dtype=np.dtype('<u2'))
print(len(sample_ecg))
sample_input = json.dumps({'data': sample_ecg.tolist()})



7500


In [6]:
local_service.run(sample_input)

NameError: name 'local_service' is not defined

# Edit score.py and reload the service if desired
The following cells are for the editing of the score.py file and applying the changes to the service

In [73]:
%%writefile source_dir/score.py
import json
import numpy as np 
from tensorflow.keras.models import load_model
import process_ecg
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path(("af-detection-Model"))
    model = load_model(model_path)

def run(request):
    try:
        data = json.loads(request)
        ecg_array = np.array(data['data'])
        processed = process_ecg.process(ecg_array)
        result = model.predict(processed)
        prediction = np.argmax(result)+1
        reject = 0
        if np.max(result[0]) < 0.97 and (np.max(result[0])-process_ecg.second_largest(result[0])) < 0.95:
           reject = 1
        return (int(prediction), int(reject))
    except Exception as e:
        error = str(e)
        return {'data' : error, "message" : 'unable to classify sample'}

Overwriting source_dir/score.py


In [74]:
local_service.reload()
print("________________________________________________________")


Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
________________________________________________________


# Deploy the Model the Cloud Using AKS clusters, using same configuration as the Local deployment 


In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aks_name = 'test-aks-1'

try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found, existing cluster, use it.')
except ComputeTargetException: 
    prov_config = AksCompute.provisioning_configuration()
    aks_target = ComputeTarget.create(workspace=ws, name=aks_name, provisioning_configuration = prov_config)
if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)
    

In [ ]:
from azureml.core.webservice import AksWebservice

ask_config = AksWebservice.deploy_configuration()
%%time
aks_service_name = 'aks-servive-1'
aks_service = local_service.deploy_to_cloud(name=aks_service_name,deployment_config=ask_config, deployment_target=aks_target)
aks_service.wait_for_deployment(show_output=True)
print(aks_service.state)

In [ ]:
import json
import numpy as np

sample_ecg = np.fromfile("./source_dir/ecg_sample.dat", dtype=np.dtype('<u2'))
print(len(sample_ecg))
sample_input = json.dumps({'data': sample_ecg.tolist()})

In [ ]:
aks_service.run(sample_input)

# Delete the service once finished
Once finished testing that the service works remove it to prevent additional costs accumulating over time.

In [ ]:
local_service.delete()

